In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.18.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


In [103]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()


###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

def show_video(video_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if video_path.startswith('http'):
        data_url = video_path
    else:
        mp4 = open(video_path, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(image_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))


In [61]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.repnet_tf.inference'
MSGKEY = 'zmq.repnet_tf.test'
TEST_SAMPLES = {
    'test1': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4',
    'test2': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/repnet_static_test.mp4',
    'test3': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/repnet_mix_test.mp4',
    'test4': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/repnet_mix2_test.mp4'
}
API_INFERENCE, API_POPMSG 

('http://116.85.5.40:9119/raceai/framework/inference',
 'http://116.85.5.40:9119/raceai/private/popmsg')

## 接口测试

### 输入

```json

{
    "task": "zmq.repnet_tf.inference",  // 任务类型, 不可修改
    "cfg": {
        "pigeon": {  // 信鸽, 由服务器自行添加字段, 异步数据输出时会携带这些数据返回, 其中"msgkey", "user_code"必须有
            "msgkey": "zmq.repnet_tf.test",  // 异步数据Redis消息队列key
            "user_code": "123", // 任务由谁触发
        },
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4", // 视频地址
        "save_video": True,   // 是否保存处理后的视频 (开启后, 速度非常慢)
        "batch_size": 20,     // batch_size
        "threshold": 0.2,     // 模型预测分数阀值
        "in_threshold": 0.5,  // 模型预测帧是否在周期内的阀值
        "strides": [1, 2, 3, 4],     // 送入模型的帧跳跃步长, 1表示逐个帧, 2表示跳跃一帧..., 模型会自行选择步长最合适的
        "constant_speed": False,     // 视频周期运动速度是否常数
        "median_filter": True,       // 是否对周期检测做中值滤波
        "fully_periodic": False      // 周期检测是否使用全周期(所有帧), 而不是由in_threshold指定的阀值过滤
    }
}
```

In [67]:
_TEMPLATE_ = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user_code": "%s",
        },
        "video": "%s",
        "save_video": True,
        "batch_size": 20,
        "threshold": 0.2,
        "in_threshold": 0.5,
        "strides": [1, 2, 3, 4],
        "constant_speed": False,
        "median_filter": True,
        "fully_periodic": False
    }
}'''

cfgdatas = [ _TEMPLATE_ % (
    MODEL_TASK, MSGKEY,
    user_code, video_url
) for user_code, video_url in TEST_SAMPLES.items()]

print("Input:\n\n", cfgdatas[0])

Input:

 {
    "task": "zmq.repnet_tf.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.repnet_tf.test",
            "user_code": "test1",
        },
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4",
        "save_video": True,
        "batch_size": 20,
        "threshold": 0.2,
        "in_threshold": 0.5,
        "strides": [1, 2, 3, 4],
        "constant_speed": False,
        "median_filter": True,
        "fully_periodic": False
    }
}


### 输出

```json
 {
    "pigeon": {
        "msgkey": "zmq.repnet_tf.test",
        "user_code": "123"
    },
    "task": "zmq.repnet_tf.inference",
    "errno": 0,
    "progress": 100.0,
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-target.mp4",
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-results.json"
}
```

In [105]:
reqdata = eval(cfgdatas[2])
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)

{'errno': 0}

In [106]:
def get_result():
    test_result_mp4 = None
    while True:
        result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
        if len(result) > 0 and 'progress' in result[-1]:
            progress = int(result[-1]['progress'])
            print_progress_bar(progress)
            if progress >= 100.0:
                print('\nOutput:\n\n', json.dumps(result[-1], indent=4))
                if 'target_mp4' in result[-1]:
                    return result[-1]
        time.sleep(3)
result = get_result()

Progress: 100%: ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋
Output:

 {
    "pigeon": {
        "msgkey": "zmq.repnet_tf.test",
        "user_code": "test3"
    },
    "task": "zmq.repnet_tf.inference",
    "errno": 0,
    "progress": 100.0,
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/test3/repnet_tf-target.mp4",
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/test3/repnet_tf-results.json"
}


In [110]:
json_result = requests.get(result['target_json']).json()
print(json.dumps(json_result, indent=4))

{
    "period": 75.07298992137551,
    "score": 0.5507646203041077,
    "stride": 2,
    "fps": 29.973997320936096,
    "num_frames": 634,
    "infer_time": 2.7443716526031494,
    "frames_period": [
        {
            "image_id": "0.jpg",
            "at_time": 0.033,
            "within_period": 0.7064347267150879,
            "pframe_counts": 0.019999999552965164,
            "cum_counts": 0.019999999552965164
        },
        {
            "image_id": "1.jpg",
            "at_time": 0.067,
            "within_period": 0.7064347267150879,
            "pframe_counts": 0.019999999552965164,
            "cum_counts": 0.03999999910593033
        },
        {
            "image_id": "2.jpg",
            "at_time": 0.1,
            "within_period": 0.8335369825363159,
            "pframe_counts": 0.019999999552965164,
            "cum_counts": 0.05999999865889549
        },
        {
            "image_id": "3.jpg",
            "at_time": 0.133,
            "within_period": 0.8335369

In [108]:
show_video(json_result['target_mp4'])

## JSON结果数据

```json
{
    "period": 58.022187843507346,      // 视频周期平均值(帧数)
    "score": 0.6532081365585327,       // 视频预测的平均分数
    "stride": 2,                       // 最佳步长
    "fps": 29.916666666666668,         // 视频帧率
    "num_frames": 634,                 // 视频帧总数
    "infer_time": 3.5414674282073975,  // 模型推理用时
    "mkvideo_time": 88.88              // 处理合成视频用时
    "frames_period": [                 // 每一帧的周期信息
        {
            "image_id": "0.jpg",                    // 第 1 帧
            "at_time": 0.033,                       // 该帧对应的视频中的时间数(秒)
            "within_period": 0.7064347267150879,    // 属于周期内的帧的置信度
            "pframe_counts": 0.019999999552965164,  // 每一帧所代表周期数量 (小数) 
            "cum_counts": 0.019999999552965164      // 循环周期次数依次累加
        },
        { 
            "image_id": "1.jpg",                    // 第 2 帧
            "at_time": 0.067,
            "within_period": 0.7064347267150879,
            "pframe_counts": 0.019999999552965164,
            "cum_counts": 0.03999999910593033
        },
        ....
        {
            "image_id": "392.jpg",                   // 第 393 帧
            "at_time": 13.111,
            "within_period": 0.7592176795005798,
            "pframe_counts": 0.02083333395421505,
            "cum_counts": 5.636037701740861
        },
        {
            "image_id": "393.jpg",
            "at_time": 13.145,
            "within_period": 0.7592176795005798,
            "pframe_counts": 0.02083333395421505,
            "cum_counts": 5.656871035695076
        },
      ....
    ],
    "mkvideo_time": 54.93384122848511,   // 视频合成耗费的时间
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/test3/repnet_tf-target.mp4" // 合成视频的存放地址
}
```

## References